In [ ]:
"""
Created on Fri Oct 27 10:34 2023

Look at temperature data from CMIP

Author: @claraburgard
"""

In [ ]:
import xarray as xr


In [ ]:
%matplotlib qt5

READ IN DATA

In [ ]:
home_path = '/bettik/burgardc/'


Models

In [ ]:
inputpath_tas = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/CMIP_TEMP/'

In [ ]:
mod_list = ['ACCESS-CM2','ACCESS-ESM1-5','CNRM-CM6-1','CNRM-ESM2-1','GISS_E2-1-H',
            'CESM2-WACCM','CESM2','CanESM5','IPSL-CM6A-LR','MRI-ESM2-0','MPI-ESM1-2-HR',
            'GFDL-CM4','GFDL-ESM4','UKESM1-0-LL']

In [ ]:
tas_mod_list = []

for mod in mod_list:
    print(mod)
    if mod not in ['GISS_E2-1-H']:
        tas_scen_list = []
        for scen in ['historical','ssp126','ssp245','ssp585']:
            if (mod == 'GFDL-CM4') and (scen == 'ssp126'):
                tas_mod_scen['tas'] = tas_mod_scen['tas']*np.nan
            else:
                tas_mod_scen = xr.open_dataset(inputpath_tas+mod+'/tas_Amon_'+mod+'_'+scen+'_fldmean_ymean.nc')
                tas_mod_scen['time'] = tas_mod_scen.time.dt.year
            tas_out = tas_mod_scen['tas'].squeeze(drop=True)
            if 'height' in tas_out.coords:
                tas_out = tas_out.drop('height')
            tas_scen_list.append(tas_out.assign_coords({'scenario':scen}))

        tas_all_scen = xr.concat(tas_scen_list,dim='scenario')
        tas_mod_list.append(tas_all_scen.assign_coords({'model':mod}))
        
tas_all_mod = xr.concat(tas_mod_list, dim='model')

In [ ]:
tas_all_scen

In [ ]:
for scen in ['historical','ssp126','ssp245','ssp585']:
    tas_all_scen.sel(scenario=scen).plot()


Observations

In [ ]:
inputpath_raw = home_path+'/DATA/SUMMER_PAPER/raw/'
HadCRUT_temp = xr.open_dataset(inputpath_raw + 'HadCRUT.5.0.0.0.analysis.summary_series.global.annual.nc')
HadCRUT_tas = HadCRUT_temp['tas_mean']
HadCRUT_tas['time'] = HadCRUT_temp.time.dt.year

In [ ]:
anom_baseline = HadCRUT_tas.sel(time=range(1850,1880)).mean('time')
anom_picontrol = HadCRUT_tas - anom_baseline

In [ ]:
anom_mod_scen = tas_all_mod - tas_all_mod.sel(scenario='historical').sel(time=range(1850,1880)).mean('time')

In [ ]:
anom_mod_scen

In [ ]:
for scen in ['historical','ssp126','ssp245','ssp585']:
    if scen == 'historical':
        ccolor='grey'
    elif scen == 'ssp126':
        ccolor='orange'
    elif scen == 'ssp245':
        ccolor='maroon' 
    elif scen == 'ssp585':
        ccolor='red'         
    for mod in anom_mod_scen.model:
        anom_mod_scen.sel(model=mod,scenario=scen).plot(color=ccolor,alpha=0.2)
anom_picontrol.plot(color='k')

Masks

In [ ]:
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)